In [ ]:
import json
from openai import OpenAI
from datetime import datetime

# Set your API key here
api_key = "sk-svcacct-WqOWrSDc4rSGBDG7-QcN4cZqh6SiGuIEPb44EL3_nhxAaoeeMpOhTLbbhA8CrwT3BlbkFJ1RwNoTMV9rjb57Ir5WtfPhMB54f7MKYGjaPgLdsPAHTplF-u-6m3Gb0R-_3-0A"  # Replace with your actual OpenAI API key
client = OpenAI(api_key=api_key)

def read_input_file(filename):
    try:
        with open(filename, 'r') as file:
            content = file.read()
            if not content.strip().startswith('{'):
                content = '{' + content
            if not content.strip().endswith('}'):
                content = content + '}'
            return json.loads(content)
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON format in file: {e}")
        return None

def chunk_data(products, chunk_size=50):
    """Split the products into smaller chunks"""
    for i in range(0, len(products), chunk_size):
        yield products[i:i + chunk_size]

def process_chunk(chunk_data):
    # Convert the chunk to a string format for the prompt
    data_str = json.dumps({"ListOfProducts": chunk_data}, indent=2)
    
    messages = [
        {"role": "system", "content": "You are a helpful assistant that analyzes product descriptions and extracts product codes."},
        {"role": "user", "content": f"""
Your task is to analyze each description and categorize it based on the **position** and **pattern** of the product code in the description.

### Task Details:
1. **Identify and Extract Product Code**:  
   - Identify the product code in each description. Typically, this appears with a prefix like `Pn:` or has a distinct pattern (e.g., alphanumeric sequences or separated by special characters).

2. **Categorize Descriptions**:  
   - Create a category for each description based on:
     - The **position** of the product code (e.g., at the end, middle, or beginning).
     - The **pattern** of the product code (e.g., numeric, alphanumeric, prefixed).
     - The **position** of the PN code like where is it present suffix or prefix or middle.
     - if the pn is numeric and at the end of the description then the category name should be "PN numeric suffix" and if the pn is alphanumeric and at the end of the description then the category name should be "PN alphanumeric suffix" and if the pn is numeric and at the beginning of the description then the category name should be "PN numeric prefix" and if the pn is alphanumeric and at the beginning of the description then the category name should be "PN alphanumeric prefix" and if the pn is numeric and in the middle of the description then the category name should be "PN numeric middle" and if the pn is alphanumeric and in the middle of the description then the category name should be "PN alphanumeric middle"
         

Input Data:
{data_str}

Please provide the results in this JSON structure:
{{
    "results": [
        {{"Description": "", "Product Code": "", "Category Name": ""}},
    ]
}}

Focus on:
1. Extracting product codes (especially after "Pn:" or unique identifiers)
2. Categorizing based on code format and position
3. Maintaining the exact format specified above
"""}
    ]
    
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error processing chunk: {e}")
        return None

def merge_results(results_list):
    """Merge all chunk results into a single JSON structure"""
    all_results = []
    for result in results_list:
        try:
            if result:
                # Parse the JSON string
                result_dict = json.loads(result)
                if "results" in result_dict:
                    all_results.extend(result_dict["results"])
        except json.JSONDecodeError as e:
            print(f"Error parsing result: {e}")
            continue
    
    return {"results": all_results}

# Main execution
print(f"Current Date and Time (UTC): {datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')}")

input_file = "partnumsample.txt"
print(f"\nReading input file: {input_file}")

# Read the data
input_data = read_input_file(input_file)
if input_data and "ListOfProducts" in input_data:
    products = input_data["ListOfProducts"]
    print(f"Total products to process: {len(products)}")
    
    # Process in chunks
    all_results = []
    for i, chunk in enumerate(chunk_data(products, chunk_size=25)):
        print(f"\nProcessing chunk {i+1}...")
        result = process_chunk(chunk)
        if result:
            all_results.append(result)
    
    # Merge all results
    if all_results:
        final_results = merge_results(all_results)
        
        # Save results
        with open("analysis_results.json", "w") as f:
            json.dump(final_results, f, indent=2)
        
        print("\nProcessing complete!")
        print(f"Total results processed: {len(final_results['results'])}")
        print("Results have been saved to 'analysis_results.json'")
        
        # Display first few results as example
        print("\nExample of first few results:")
        for item in final_results['results'][:3]:
            print(json.dumps(item, indent=2))
else:
    print("No valid data to process")

Current Date and Time (UTC): 2024-12-27 11:08:59

Reading input file: partnumsample.txt
Total products to process: 460

Processing chunk 1...

Processing chunk 2...

Processing chunk 3...

Processing chunk 4...

Processing chunk 5...

Processing chunk 6...

Processing chunk 7...

Processing chunk 8...

Processing chunk 9...

Processing chunk 10...

Processing chunk 11...

Processing chunk 12...

Processing chunk 13...

Processing chunk 14...

Processing chunk 15...

Processing chunk 16...

Processing chunk 17...

Processing chunk 18...

Processing chunk 19...
Error parsing result: Expecting value: line 1 column 1 (char 0)
Error parsing result: Expecting value: line 1 column 1 (char 0)
Error parsing result: Expecting value: line 1 column 1 (char 0)
Error parsing result: Expecting value: line 1 column 1 (char 0)
Error parsing result: Expecting value: line 1 column 1 (char 0)
Error parsing result: Expecting value: line 1 column 1 (char 0)
Error parsing result: Expecting value: line 1 colu